In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
df = pd.read_excel('raw_data_with_label.xlsx', sheet_name='links', index_col=0)
df.head()

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike,y
0,https://www.youtube.com/watch?v=-0WQnwNFqJM,AI learns to Speedrun QWOP using Machine Learning,Wesley Liao,https://www.youtube.com/channel/UCair-JZ6XmtJk...,226613.0,25 de fev. de 2021,"2.729 marcações ""Gostei""","33 marcações ""Não gostei""",0
1,https://www.youtube.com/watch?v=-5hEYRt8JE0,How to Become A Machine Learning Engineer | Ho...,#MachineLearningAlgorithms,https://www.youtube.com/hashtag/machinelearnin...,53263.0,3 de set. de 2018,"1.407 marcações ""Gostei""","27 marcações ""Não gostei""",0
2,https://www.youtube.com/watch?v=-6RqxhNO2yY,Python For Data Science Full Course - 9 Hours ...,#edureka,https://www.youtube.com/hashtag/edureka,142715.0,15 de mar. de 2020,"3.990 marcações ""Gostei""","57 marcações ""Não gostei""",0
3,https://www.youtube.com/watch?v=-AkBfBWr_Gw,Can You Become a Data Scientist?,#data,https://www.youtube.com/hashtag/data,776910.0,14 de ago. de 2018,"12.316 marcações ""Gostei""","431 marcações ""Não gostei""",0
4,https://www.youtube.com/watch?v=-DDrE6T0ct4,Binary Image Classification with CNN (Deep Lea...,#download,https://www.youtube.com/hashtag/download,2290.0,23 de mar. de 2020,"66 marcações ""Gostei""","Sem marcações ""Não gostei""",1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1768 entries, 0 to 1767
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         1768 non-null   object 
 1   title        1741 non-null   object 
 2   canal_nome   1748 non-null   object 
 3   canal_link   1748 non-null   object 
 4   view_counts  1747 non-null   float64
 5   video_date   1461 non-null   object 
 6   like         1711 non-null   object 
 7   dislike      1711 non-null   object 
 8   y            1768 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 138.1+ KB


In [4]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1452 entries, 0 to 1766
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         1452 non-null   object 
 1   title        1452 non-null   object 
 2   canal_nome   1452 non-null   object 
 3   canal_link   1452 non-null   object 
 4   view_counts  1452 non-null   float64
 5   video_date   1452 non-null   object 
 6   like         1452 non-null   object 
 7   dislike      1452 non-null   object 
 8   y            1452 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 113.4+ KB


In [5]:
df.dropna(inplace=True)

In [6]:
df.y.value_counts()

0    1251
1     201
Name: y, dtype: int64

In [7]:
df.shape

(1452, 9)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [16]:
df_limpo = pd.DataFrame(index=df.index)

## 1. Limpeza da data

In [17]:
clean_date = df['video_date'].str.extract(r'(\d+) de (\w+)\. de (\d+)')
clean_date

,0,1,2
0,25,fev,2021
1,3,set,2018
2,15,mar,2020
3,14,ago,2018
4,23,mar,2020
...,...,...,...
1759,24,jan,2019
1760,30,set,2020
1762,9,dez,2019
1765,12,mar,2021


In [18]:
clean_date[0] = clean_date[0].map(lambda x: "0"+ x[0] if len(x[0]) == 1 else x)
clean_date.head()

,0,1,2
0,02,fev,2021
1,03,set,2018
2,01,mar,2020
3,01,ago,2018
4,02,mar,2020


In [19]:
mapa_meses = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: ' '.join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format='%d %b %Y')

In [23]:
df_limpo.head(5)

,date
0,2021-02-02
1,2018-09-03
2,2020-03-01
3,2018-08-01
4,2020-03-02
